<h2><center>NLP Text Classification</h2>

## I. Introduction

### 1.1 Domain-specific area
This project provides an analysis of textual data on Twitter to accurately detect and classify threatening or harmful content using sentiment analysis techniques. This would provide the cybersecurity industry a tool that takes in a corpus of text for training to develop a strong detection system.

### 1.2 Objectives
Due to popular algorithms being centered around the detection of cyberbullying on social media (Cynthia Van Hee et al., 2018), it is important for this project to widen the scope of detection. While the general detection algorithms focus mainly on terrorism and cyberbullying, it is a known fact that cybersecurity encompasses more than those 2 focuses. (Khairy, Mahmoud and Abd-El-Hafeez, 2021) While full security and safety of users cannot be ensured, making these adjustments would contribute valuable insights for future development.

### 1.3 Dataset
To begin this project, an extensive amount of textual data corpora is required. After researching large datasets of Tweets, Sentiment140 Kaggle was proven to be the best for this project. With 1.6 million tweets extracted using the Twitter API, the authors have categorised each tweet to have either a positive, neutral or negative sentiment, which is beneficial for the algorithm in categorising harmful texts.

The dataset consists of the target (defined as the sentiment of the text), the tweet IDs, date, flags (possible queries, which would be removed in the initialisation phase of extracting the data), the username, and the text of the tweet.

### 1.4 Evaluation methodology

## II. Implementation

### 2.1 Pre-processing
(writeup not needed)
<br>Convert/store the dataset locally and preprocess the data. Describe the text representation
(e.g., bag of words, word embedding, etc.) and any pre-processing steps you have applied
and why they were needed (e.g. tokenization, lemmatization). Describe the vocabulary and
file type/format, e.g. CSV file.

#### Acquiring dataset
The dataset on the collection of Tweets were acquired from Kaggle by downloading the CSV file. The author of this dataset is Μαριος Μιχαηλιδης KazAnova. The code for importing the dataset is shown below:

#### 2.1.1 Importing libraries
- <b>pandas library</b> was imported to process and handle dataframes in Python. It is used to help write and read from CSV files while handling real-world messy data and processing them into a proper format

- <b>numpy library</b> was imported to handle calculations and use numpy arrays for statistical calculations

- <b>matplotlib library</b> was imported to plot the data and represent it graphically

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#### 2.1.2 Creating helper functions
- <b>displaysetsH</b> takes in a list of datasets and an optional number of rows to display the head of each dataset

- <b>displaysetsT</b> takes in a list of datasets and an optional number of rows to display the tail of each dataset

- <b>resetidx</b> takes in a list of datasets to reset the indexes of each dataset

In [2]:
def displaysetsH(datasets, amt = 5):
    for dataset in datasets:
        display(dataset.head(amt))
        
def displaysetsT(datasets, amt = 5):
    for dataset in datasets:
        display(dataset.head(amt))
        
def resetidx(datasets):
    for dataset in datasets:
        dataset.reset_index(drop = True, inplace = True)

#### 2.1.3 Importing Dataset

In [ ]:
tweets_df = pd.read_csv()

### 2.2 Baseline performance
(writeup not needed)

### 2.3 Classification approach
(writeup not needed)

### 2.4 Coding style
(writeup not needed)

## III. Conclusions

### 3.1 Evaluation

### 3.2 Summary and conclusions

## Temporary reference list
* to use citation generator

- Cynthia Van Hee, Jacobs, G., Emmery, C., Desmet, B., Lefever, E., Verhoeven, B., Guy De Pauw, Daelemans, W. and Hoste, V. (2018). Automatic detection of cyberbullying in social media text. PLOS ONE, [online] 13(10), p.e0203794. doi:https://doi.org/10.1371/journal.pone.0203794.
- Khairy, M., Mahmoud, T.M. and Abd-El-Hafeez, T. (2021). Automatic Detection of Cyberbullying and Abusive Language in Arabic Content on Social Networks: A Survey. Procedia Computer Science, [online] 189, pp.156–166. doi:https://doi.org/10.1016/j.procs.2021.05.080.